# Task2 
## tashk 2.1 y 2.2: 
Definir las variables y restringir días así como la relación para los estudiantes y los cursos que tienen

In [4]:
import random
import time
import math

# Definir las variables y dominios
examenes = ["Quimica", "Biología", "Matemáticas", "Física", "Algoritmos", "Diseño de datos", "Literatura"]
dias = ["Lunes", "Martes", "Miercoles"]

# Relaciones entre estudiantes y exámenes
estudiantes_examenes = [
    ("Estudiante1", "Quimica"),
    ("Estudiante1", "Biología"),
    ("Estudiante2", "Biología"),
    ("Estudiante2", "Matemáticas"),
    ("Estudiante3", "Matemáticas"),
    ("Estudiante3", "Física"),
    ("Estudiante4", "Física"),
    ("Estudiante4", "Algoritmos"),
    ("Estudiante1", "Diseño de datos"),
    ("Estudiante2", "Diseño de datos"),
    ("Estudiante3", "Literatura"),
    ("Estudiante4", "Literatura"),
]

### 2.3 backtraking search

In [6]:
def backtracking_search(examenes, estudiantes_examenes, dias):
    def backtrack(asignacion):
        # Verificar si la asignación actual es completa
        if len(asignacion) == len(examenes):
            return asignacion
        
        # Seleccionar el próximo examen a asignar
        examen_actual = None
        for examen in examenes:
            if examen not in asignacion:
                examen_actual = examen
                break
        
        # Probar cada día posible para el examen actual
        for dia in dias:
            # Verificar si el día satisface las restricciones
            if dia_valido(examen_actual, dia, asignacion, estudiantes_examenes):
                # Asignar el examen al día actual
                asignacion[examen_actual] = dia
                
                # Realizar una búsqueda recursiva
                resultado = backtrack(asignacion)
                if resultado is not None:
                    return resultado
                
                # Si no se encontró una solución válida, deshacer la asignación
                del asignacion[examen_actual]
        
        # No se encontró una asignación válida para el examen actual
        return None
    
    # Función para verificar si un día satisface las restricciones
    def dia_valido(examen, dia, asignacion, estudiantes_examenes):
        # Verificar restricción: cada examen debe programarse en un día diferente
        if dia in asignacion.values():
            return False
        
        # Verificar restricción: ningún estudiante puede tener más de un examen por día
        for estudiante, examen_estudiante in estudiantes_examenes:
            if examen_estudiante == examen and estudiante in asignacion and asignacion[estudiante] == dia:
                return False
        
        # Verificar restricción: los estudiantes que toman el mismo curso no pueden tener exámenes programados para el mismo día
        for estudiante, examen_estudiante in estudiantes_examenes:
            if examen_estudiante == examen and estudiante in asignacion and asignacion[estudiante] == dia:
                return False
        
        return True
    
    # Iniciar la búsqueda con una asignación vacía
    return backtrack({})

solucion = backtracking_search(examenes, estudiantes_examenes, dias)
print(solucion)


None


### 2.4 beam search algorithm

In [8]:
def beam_search(examenes, estudiantes_examenes, dias, beam_width):
    # Inicializar el conjunto de soluciones con una solución vacía
    soluciones = [{}]
    
    # Iterar a través de cada examen
    for examen in examenes:
        nuevas_soluciones = []
        
        # Expandir cada solución candidata en el conjunto actual
        for asignacion in soluciones:
            # Probar cada día posible para el examen actual
            for dia in dias:
                # Verificar si el día satisface las restricciones
                if dia_valido(examen, dia, asignacion, estudiantes_examenes):
                    # Crear una nueva asignación basada en la asignación actual y el nuevo examen
                    nueva_asignacion = asignacion.copy()
                    nueva_asignacion[examen] = dia
                    
                    # Agregar la nueva asignación al conjunto de nuevas soluciones
                    nuevas_soluciones.append(nueva_asignacion)
        
        # Seleccionar las mejores soluciones basadas en una heurística (cantidad de restricciones incumplidas)
        soluciones = sorted(nuevas_soluciones, key=lambda x: contar_restricciones_incumplidas(x, estudiantes_examenes))[:beam_width]
    
    # Devolver la mejor solución encontrada
    mejor_solucion = min(soluciones, key=lambda x: contar_restricciones_incumplidas(x, estudiantes_examenes))
    return mejor_solucion

# Función para verificar si un día satisface las restricciones
def dia_valido(examen, dia, asignacion, estudiantes_examenes):
    # Verificar restricción: Todos los exámenes en días diferentes
    if dia in asignacion.values():
        return False
    
    # Verificar restricción: Ningún estudiante tiene más de un examen por día
    estudiantes_con_examen = [estudiante for estudiante, examen_asignado in asignacion.items() if examen_asignado == dia]
    if len(estudiantes_con_examen) > 1:
        return False
    
    # Verificar restricción: Los estudiantes que toman el mismo curso no pueden tener exámenes programados para el mismo día
    for estudiante, examen_asignado in asignacion.items():
        if (estudiante, examen) in estudiantes_examenes and (estudiante, examen_asignado) in estudiantes_examenes:
            return False
    
    return True

# Función para contar la cantidad de restricciones incumplidas en una asignación
def contar_restricciones_incumplidas(asignacion, estudiantes_examenes):
    restricciones_incumplidas = 0
    
    # Verificar restricción: Todos los exámenes en días diferentes
    dias_asignados = list(asignacion.values())
    dias_unicos = set(dias_asignados)
    restricciones_incumplidas += len(dias_asignados) - len(dias_unicos)
    
    # Verificar restricción: Ningún estudiante tiene más de un examen por día
    for dia in dias_unicos:
        estudiantes_con_examen = [estudiante for estudiante, examen_asignado in asignacion.items() if examen_asignado == dia]
        restricciones_incumplidas += max(0, len(estudiantes_con_examen) - 1)
    
    # Verificar restricción: Los estudiantes que toman el mismo curso no pueden tener exámenes programados para el mismo día
    for estudiante, examen_asignado in asignacion.items():
        for otro_estudiante, otro_examen in asignacion.items():
            if estudiante != otro_estudiante and (estudiante, otro_examen) in estudiantes_examenes and (otro_estudiante, examen_asignado) in estudiantes_examenes:
                restricciones_incumplidas += 1
    
    return restricciones_incumplidas

In [ ]:
beam_width = 3  # Ancho del haz (beam width)

# Ejemplo de uso
mejor_solucion_beam = beam_search(examenes, estudiantes_examenes, dias, beam_width)
print("Mejor solución encontrada por Beam Search:")
print(mejor_solucion_beam)

### 2.5 local search algorithm

In [5]:
# Asignar un día inicial aleatorio a cada examen
solucion_inicial = {examen: random.choice(dias) for examen in examenes}

# Definir la función de restricciones
def restricciones(solucion):
    # Comprobar que todos los exámenes se realicen en días diferentes
    if len(set(solucion.values())) < len(examenes):
        return False
    
    # Comprobar que ningún estudiante tenga más de un examen por día
    for estudiante in set(estudiante for estudiante, examen in estudiantes_examenes):
        dias_examenes_estudiante = [solucion[examen] for estudiante_, examen in estudiantes_examenes if estudiante_ == estudiante]
        if len(dias_examenes_estudiante) != len(set(dias_examenes_estudiante)):
            return False
    
    # Comprobar que los estudiantes que toman el mismo curso no tengan exámenes el mismo día
    for estudiante1, Quimica in estudiantes_examenes:
        for estudiante2, Biología in estudiantes_examenes:
            if Quimica == Biología and estudiante1 != estudiante2:
                if solucion[Quimica] == solucion[Biología]:
                    return False

    return True

# Función de evaluación para medir qué tan buena es una solución
def evaluar_solucion(solucion):
    violaciones = 0
    for estudiante in set(estudiante for estudiante, examen in estudiantes_examenes):
        dias_examenes_estudiante = [solucion[examen] for estudiante_, examen in estudiantes_examenes if estudiante_ == estudiante]
        violaciones += len(dias_examenes_estudiante) - len(set(dias_examenes_estudiante))
    return violaciones

# Algoritmo de búsqueda local mejorado con enfriamiento simulado
def local_search_enhanced(solucion):
    iteraciones = 0
    inicio = time.time()
    temperatura = 1.0
    tasa_enfriamiento = 0.95
    mejor_solucion = solucion.copy()
    mejor_evaluacion = evaluar_solucion(mejor_solucion)
    while temperatura > 0.01 and iteraciones < 10000:
        examen_aleatorio = random.choice(examenes)
        dia_aleatorio = random.choice(dias)
        solucion_anterior = solucion.copy()
        solucion[examen_aleatorio] = dia_aleatorio
        evaluacion_actual = evaluar_solucion(solucion)
        if evaluacion_actual < mejor_evaluacion:
            mejor_solucion = solucion.copy()
            mejor_evaluacion = evaluacion_actual
        elif math.exp((mejor_evaluacion - evaluacion_actual) / temperatura) > random.random():
            solucion = solucion_anterior
        temperatura *= tasa_enfriamiento
        iteraciones += 1
    fin = time.time()
    tiempo_ejecucion = fin - inicio
    return mejor_solucion, tiempo_ejecucion

# Solución
solucion_final, tiempo_ejecucion = local_search_enhanced(solucion_inicial)
print("Solución final:", solucion_final)
print("Tiempo de ejecución:", tiempo_ejecucion, "segundos")


Solución final: {'Quimica': 'Miercoles', 'Biología': 'Martes', 'Matemáticas': 'Martes', 'Física': 'Miercoles', 'Algoritmos': 'Martes', 'Diseño de datos': 'Miercoles', 'Literatura': 'Lunes'}
Tiempo de ejecución: 0.0009748935699462891 segundos


El algoritmo de búsqueda local demostró ser rápido, con un tiempo de ejecución de aproximadamente 0.000 segundos para el problema de programación de exámenes,fue bastante óptimo encontrando la solución.